### **Carga o Adquisición de Datos**
---
Tiempo estimado ~ 6 horas

Espacio Requerido ~ 10GB

In [16]:
import requests
import json
import time
import base64
import zipfile
import io
import pandas as pd
import os
from tqdm import tqdm
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import numpy as np
from datetime import datetime
import multiprocessing as mp
from functools import partial

In [12]:
# Navegar hasta la raíz del proyecto 
WORKSPACE = os.path.abspath(os.path.join(os.getcwd(), '../../'))

# Definir la ruta de la carpeta de datos
DATA_FOLDER = os.path.join(WORKSPACE, 'data')

print("Workspace:", WORKSPACE)
print("Data folder:", DATA_FOLDER)

Workspace: /home/jovyan/work/proyecto-mlds6
Data folder: /home/jovyan/work/proyecto-mlds6/data


In [13]:
def get_station_data(station_codes, start_date, end_date, id_parametro, etiqueta, group_size=20):
    """
    Obtiene datos de estaciones del servidor IDEAM.

    Entradas:
    - station_codes: Lista de códigos de estaciones
    - start_date: Fecha de inicio
    - end_date: Fecha de fin
    - id_parametro: ID del parámetro a obtener
    - etiqueta: Etiqueta del parámetro
    - group_size: Tamaño del grupo de estaciones para procesar

    Salida:
    - None (guarda los archivos en DATA_FOLDER)
    """
    url = "http://dhime.ideam.gov.co/server/rest/services/AtencionCiudadano/DescargarArchivo/GPServer/DescargarArchivo/submitJob"
    groups = [station_codes[i:i + group_size] for i in range(0, len(station_codes), group_size)]

    empty_groups = 0
    overtime_groups = 0
    total_groups = len(groups)

    with tqdm(total=total_groups, desc=f"PROCESANDO {etiqueta} ({id_parametro})", unit="grupo") as pbar:
        for group in groups:
            filter_stations = "~or~".join([f"(IdParametro~eq~'{id_parametro}'~and~Etiqueta~eq~'{etiqueta}'~and~IdEstacion~eq~'{code}')" for code in group])
            params = {
                "Filtro": f"sort=&filter=({filter_stations})&group=&fechaInicio={start_date}T05%3A00%3A00.000Z&fechaFin={end_date}T05%3A00%3A00.000Z&mostrarGrado=true&mostrarCalificador=true&mostrarNivelAprobacion=true",
                "Items": json.dumps([{"IdParametro": id_parametro, "Etiqueta": etiqueta, "EsEjeY1": False, "EsEjeY2": False, "EsTipoLinea": False, "EsTipoBarra": False, "TipoSerie": "Estandard", "Calculo": ""}] * len(group)),
                "f": "pjson"
            }

            response = requests.post(url, data=params)

            if response.status_code == 200:
                job_id = response.json()['jobId']
                zip_url = f"http://dhime.ideam.gov.co/server/rest/services/AtencionCiudadano/DescargarArchivo/GPServer/DescargarArchivo/jobs/{job_id}/results/Archivo?f=pjson"

                start_time = time.time()
                saved_data = False
                while True:
                    zip_response = requests.get(zip_url)
                    if zip_response.status_code == 200 and 'value' in zip_response.json():
                        base64_string = zip_response.json()['value']
                        padding = 4 - (len(base64_string) % 4)
                        if padding:
                            base64_string += '=' * padding

                        try:
                            decoded_bytes = base64.b64decode(base64_string)
                            with zipfile.ZipFile(io.BytesIO(decoded_bytes)) as zip_file:
                                for filename in zip_file.namelist():
                                    if filename.endswith('.csv'):
                                        with zip_file.open(filename) as f:
                                            csv_data = f.read()
                                        os.makedirs(f'{DATA_FOLDER}/variables/', exist_ok=True)
                                        with open(f'{DATA_FOLDER}/variables/{etiqueta}.csv', 'ab') as file:
                                            file.write(csv_data)
                                        saved_data = True
                            break
                        except Exception as e:
                            if not saved_data:
                                empty_groups += 1
                            break
                    elif time.time() - start_time > 120:
                        if not saved_data:
                            overtime_groups += 1
                            print(f"{etiqueta} : {group}")
                        break
                    else:
                        time.sleep(1)

            pbar.update(1)

    print(f"Número de grupos sin datos: {empty_groups}/{total_groups}")
    print(f"Número de grupos fuera del límite de espera: {overtime_groups}/{total_groups}")

In [ ]:
def procesar_estacion(codigo_estacion, variables_dataframes, fecha_inicio, fecha_fin, output_folder):
    """
    Procesa una estación individual y crea su archivo CSV con todas las variables.
    
    Args:
        codigo_estacion (str): Código de la estación a procesar
        variables_dataframes (dict): Diccionario con los DataFrames de variables
        fecha_inicio (str): Fecha de inicio del período a procesar
        fecha_fin (str): Fecha fin del período a procesar
        output_folder (str): Carpeta donde se guardarán los archivos CSV
    """
    try:
        # Crear un DataFrame vacío con el rango de fechas completo
        fecha_idx = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')
        df_estacion = pd.DataFrame(index=fecha_idx)
        df_estacion.index.name = 'Fecha'
        
        # Para cada variable, obtener los valores correspondientes a la estación
        for nombre_var, df_var in variables_dataframes.items():
            # Filtrar datos para esta estación
            datos_var = df_var[df_var['CodigoEstacion'] == codigo_estacion].copy()
            
            # Convertir los datos a serie temporal con la fecha como índice
            if not datos_var.empty:
                serie_temporal = pd.Series(
                    datos_var[nombre_var].values,
                    index=datos_var['Fecha'],
                    name=nombre_var
                )
            else:
                # Si no hay datos para esta variable, crear una serie vacía con NaN
                serie_temporal = pd.Series(
                    np.nan,
                    index=fecha_idx,
                    name=nombre_var
                )
            
            # Unir con el DataFrame principal
            # Esto automáticamente alineará las fechas y pondrá NaN donde no haya datos
            df_estacion = df_estacion.join(serie_temporal)
        
        # Eliminar las filas donde todas las variables son NaN
        df_estacion = df_estacion.dropna(how='all')

        # Verificar si tenemos al menos una variable con datos
        if len(df_estacion) > 0:
            # Guardar el DataFrame como CSV
            output_path = os.path.join(output_folder, f'{codigo_estacion}.csv')
            df_estacion.to_csv(output_path, date_format='%Y-%m-%d')
            return f"Procesada estación {codigo_estacion}"
        else:
            return f"Estación {codigo_estacion} no tiene datos para ninguna variable"
    
    except Exception as e:
        return f"Error procesando estación {codigo_estacion}: {str(e)}"


In [ ]:
def procesar_estaciones_paralelo(catalogo_estaciones_activas, variables_dataframes, 
                                 fecha_inicio, fecha_fin,
                                 output_folder, chunk_size=500):
    """
    Procesa todas las estaciones en paralelo usando todos los núcleos disponibles,
    procesando en lotes de 100 estaciones y mostrando el progreso con una barra de progreso.
    
    Args:
        catalogo_estaciones_activas (pd.DataFrame): DataFrame con las estaciones activas.
        variables_dataframes (dict): Diccionario con los DataFrames de variables.
        fecha_inicio (str): Fecha de inicio del período a procesar.
        fecha_fin (str): Fecha fin del período a procesar.
        output_folder (str): Carpeta donde se guardarán los archivos CSV.
    """
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    # Obtener lista de códigos de estaciones
    codigos_estaciones = catalogo_estaciones_activas['CODIGO'].unique()
    
    # Dividir las estaciones en lotes
    lotes_estaciones = [codigos_estaciones[i:i + chunk_size] for i in range(0, len(codigos_estaciones), chunk_size)]
    
    # Configurar el procesamiento en paralelo
    num_cores = mp.cpu_count()
    print(f"Utilizando {num_cores} núcleos para el procesamiento")
    print(f"Procesando {len(codigos_estaciones)} estaciones en {len(lotes_estaciones)} lotes de {chunk_size} estaciones")
    print(f"Variables disponibles: {list(variables_dataframes.keys())}")
    
    # Crear función parcial con los argumentos comunes
    func = partial(procesar_estacion, 
                   variables_dataframes=variables_dataframes,
                   fecha_inicio=fecha_inicio,
                   fecha_fin=fecha_fin,
                   output_folder=output_folder)
    
    # Inicializar los contadores globales para el resumen final
    estaciones_con_datos_global = 0
    estaciones_sin_datos_global = 0
    estaciones_con_error_global = 0
    
    # Inicializar barra de progreso
    with tqdm(total=len(codigos_estaciones)) as pbar:
        # Procesar los lotes en paralelo
        for lote in lotes_estaciones:
            # Crear pool de procesos
            with mp.Pool(num_cores) as pool:
                resultados = pool.map(func, lote)
            
            # Actualizar la barra de progreso
            pbar.update(len(lote))
            
            # Contadores parciales
            estaciones_con_datos = 0
            estaciones_sin_datos = 0
            estaciones_con_error = 0
            
            # Mostrar resultados parciales y actualizar contadores globales
            for resultado in resultados:
                if "no tiene datos" in resultado:
                    estaciones_sin_datos += 1
                    estaciones_sin_datos_global += 1
                elif "Error" in resultado:
                    estaciones_con_error += 1
                    estaciones_con_error_global += 1
                else:
                    estaciones_con_datos += 1
                    estaciones_con_datos_global += 1

    # Mostrar resumen final
    print("\nResumen final del procesamiento de todas las estaciones:")
    print(f"- Estaciones procesadas con éxito: {estaciones_con_datos_global}")
    print(f"- Estaciones sin datos: {estaciones_sin_datos_global}")
    print(f"- Estaciones con errores: {estaciones_con_error_global}")
    print(f"\nLos archivos CSV se encuentran en: {output_folder}")

In [ ]:
def consolidar_csv(data_path, output_file):
    """
    Lee todos los archivos CSV en una carpeta, los une y agrega una nueva columna con el nombre del archivo.
    
    Args:
        data_path (str): Ruta de la carpeta que contiene los archivos CSV.
        output_file (str): Nombre del archivo CSV de salida con los datos consolidados.
    """
    # Obtener la lista de archivos CSV en la carpeta
    archivos_csv = [f for f in os.listdir(data_path) if f.endswith('.csv')]
    
    # Lista para almacenar los DataFrames
    dataframes = []
    
    # Iterar sobre cada archivo CSV
    for archivo in archivos_csv:
        archivo_path = os.path.join(data_path, archivo)
        
        # Leer el archivo CSV en un DataFrame
        df = pd.read_csv(archivo_path)
        
        # Agregar una nueva columna con el nombre del archivo 
        df.insert(0, 'CodigoEstacion', os.path.splitext(archivo)[0])
        
        # Agregar el DataFrame a la lista
        dataframes.append(df)
    
    # Concatenar todos los DataFrames en uno solo
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    
    # Guardar el DataFrame consolidado en un nuevo archivo CSV
    df_consolidado.to_csv(output_file, index=False)
    
    print(f"Archivo CSV consolidado guardado en: {output_file}")

In [18]:
def download_cne():
    url = "https://bart.ideam.gov.co/cneideam/CNE_IDEAM.xls"
    os.makedirs(DATA_FOLDER, exist_ok=True)
    file_name = os.path.basename(url)
    file_path = os.path.join(DATA_FOLDER, file_name)
    response = requests.get(url, verify=False)

    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
    else:
        pass

In [19]:
# Leer el catálogo nacional de estaciones del IDEAM
download_cne()
catalogo_estaciones = pd.read_excel(os.path.join(DATA_FOLDER, 'CNE_IDEAM.xls'))

# Filtrar estacioness
catalogo_estaciones_activas = catalogo_estaciones[catalogo_estaciones["ESTADO"] != "Suspendida"]
print(f'Número de estaciones activas: {len(catalogo_estaciones_activas)}')

Número de estaciones activas: 2624


In [ ]:
# Uso funcion get_station_data
parametros_etiquetas = {
    "Nivel Máximo": {"IdParametro": "NIVEL", "Etiqueta": "NV_MX_D"},
    "Nivel Mínimo": {"IdParametro": "NIVEL", "Etiqueta": "NV_MN_D"},
    "Nivel Medio": {"IdParametro": "NIVEL", "Etiqueta": "NV_MEDIA_D"},
    "Temperatura Máxima": {"IdParametro": "TEMPERATURA", "Etiqueta": "TMX_CON"},
    "Temperatura Mínima": {"IdParametro": "TEMPERATURA", "Etiqueta": "TMN_CON"},
    "Humedad Relativa Máxima": {"IdParametro": "HUM RELATIVA", "Etiqueta": "HR_CAL_MX_D"},
    "Humedad Relativa Mínima": {"IdParametro": "HUM RELATIVA", "Etiqueta": "HR_CAL_MN_D"},
    #"Nubosidad 7:00 13:00 18:00": {"IdParametro": "NUBOSIDAD", "Etiqueta": "NB_CON"},
    "Precipitación Acumulada": {"IdParametro": "PRECIPITACION", "Etiqueta": "PTPM_CON"}
}

for nombre_parametro, valores in parametros_etiquetas.items():
    try:
        get_station_data(
            catalogo_estaciones_activas['CODIGO'].tolist(),
            '2000-1-1',
            '2023-12-31',
            valores['IdParametro'],
            valores['Etiqueta']
        )
    except Exception as e:
        pass

PROCESANDO NV_MEDIA_D (NIVEL): 100%|██████████| 132/132 [38:50<00:00, 17.65s/grupo]


Número de grupos sin datos: 11/132
Número de grupos fuera del límite de espera: 0/132


PROCESANDO TMN_CON (TEMPERATURA): 100%|██████████| 132/132 [28:37<00:00, 13.01s/grupo]


Número de grupos sin datos: 23/132
Número de grupos fuera del límite de espera: 0/132


PROCESANDO HR_CAL_MX_D (HUM RELATIVA): 100%|██████████| 132/132 [29:53<00:00, 13.59s/grupo]


Número de grupos sin datos: 26/132
Número de grupos fuera del límite de espera: 0/132


PROCESANDO HR_CAL_MN_D (HUM RELATIVA): 100%|██████████| 132/132 [29:03<00:00, 13.21s/grupo]


Número de grupos sin datos: 26/132
Número de grupos fuera del límite de espera: 0/132


PROCESANDO PTPM_CON (PRECIPITACION): 100%|██████████| 132/132 [1:11:54<00:00, 32.69s/grupo]

Número de grupos sin datos: 3/132
Número de grupos fuera del límite de espera: 0/132


In [39]:
# Ejemplo dataset variable precipitacion acumulada diaria
df = pd.read_csv(os.path.join(DATA_FOLDER, 'variables', 'PTPM_CON.csv'))
df.head()

,CodigoEstacion,NombreEstacion,Latitud,Longitud,Altitud,Categoria,Entidad,AreaOperativa,Departamento,Municipio,...,FechaSuspension,IdParametro,Etiqueta,DescripcionSerie,Frecuencia,Fecha,Valor,Grado,Calificador,NivelAprobacion
0,21115180,HACIENDA MANILA - AUT [21115180],3.133056,-75.081528,600,Agrometeorológica,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 04 - Huila-Caquetá,Huila,Baraya,...,NaN,PRECIPITACION,PTPM_CON,Día pluviométrico (convencional),Diaria,2000-01-03 00:00,0.0,50,NaN,1200
1,21115180,HACIENDA MANILA - AUT [21115180],3.133056,-75.081528,600,Agrometeorológica,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 04 - Huila-Caquetá,Huila,Baraya,...,NaN,PRECIPITACION,PTPM_CON,Día pluviométrico (convencional),Diaria,2000-01-04 00:00,0.0,50,NaN,1200
2,21115180,HACIENDA MANILA - AUT [21115180],3.133056,-75.081528,600,Agrometeorológica,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 04 - Huila-Caquetá,Huila,Baraya,...,NaN,PRECIPITACION,PTPM_CON,Día pluviométrico (convencional),Diaria,2000-01-05 00:00,0.0,50,NaN,1200
3,21115180,HACIENDA MANILA - AUT [21115180],3.133056,-75.081528,600,Agrometeorológica,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 04 - Huila-Caquetá,Huila,Baraya,...,NaN,PRECIPITACION,PTPM_CON,Día pluviométrico (convencional),Diaria,2000-01-06 00:00,0.0,50,NaN,1200
4,21115180,HACIENDA MANILA - AUT [21115180],3.133056,-75.081528,600,Agrometeorológica,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 04 - Huila-Caquetá,Huila,Baraya,...,NaN,PRECIPITACION,PTPM_CON,Día pluviométrico (convencional),Diaria,2000-01-07 00:00,14.6,50,NaN,1200


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51130 entries, 0 to 51129
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CodigoEstacion    51130 non-null  int64  
 1   NombreEstacion    51130 non-null  object 
 2   Latitud           51130 non-null  float64
 3   Longitud          51130 non-null  float64
 4   Altitud           51130 non-null  int64  
 5   Categoria         51130 non-null  object 
 6   Entidad           51130 non-null  object 
 7   AreaOperativa     51130 non-null  object 
 8   Departamento      51130 non-null  object 
 9   Municipio         51130 non-null  object 
 10  FechaInstalacion  51130 non-null  object 
 11  FechaSuspension   0 non-null      float64
 12  IdParametro       51130 non-null  object 
 13  Etiqueta          51130 non-null  object 
 14  DescripcionSerie  51130 non-null  object 
 15  Frecuencia        51130 non-null  object 
 16  Fecha             51130 non-null  object

In [ ]:
# Lista de DataFrames 
variables_dataframes = {}

# Leer cada archivo en la carpeta de datos
for archivo in os.listdir(os.path.join(DATA_FOLDER, 'variables')):
    if archivo.endswith('.csv'):
        # Extraer el nombre del archivo sin extensión
        nombre_variable = os.path.splitext(archivo)[0]
        
        # Leer el archivo CSV y solo cargar las columnas necesarias, especificando los tipos de datos
        archivo_path = os.path.join(DATA_FOLDER, 'variables', archivo)

        df = pd.read_csv(archivo_path, 
                            usecols=['CodigoEstacion', 'Fecha', 'Valor'],
                            dtype={'CodigoEstacion': 'int', 'Fecha': 'str', 'Valor': 'str'})  
         
        # Convertir la columna Fecha al tipo datetime forzando errores a NaN
        df['Fecha'] =  pd.to_datetime(df['Fecha'], errors='coerce').dt.normalize()  

        # Convertir la columna 'Valor' a float32, forzando errores a NaN
        df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce').astype('float32')
        
        # Redondear la columna 'Valor' a 1 decimales
        df['Valor'] = df['Valor'].round(1)
        
        # Renombrar la columna 'Valor' con el nombre de la variable (archivo)
        df.rename(columns={'Valor': nombre_variable}, inplace=True)

        # Guardar el DataFrame en el diccionario usando el nombre de la variable
        variables_dataframes[nombre_variable] = df

In [ ]:
# Crear y ejecutar el procesamiento en paralelo
procesar_estaciones_paralelo(
    catalogo_estaciones_activas=catalogo_estaciones_activas,
    variables_dataframes=variables_dataframes,
    fecha_inicio='2000-01-01',
    fecha_fin='2023-12-31',
    output_folder= os.path.join(DATA_FOLDER, 'estaciones')
)

In [ ]:
consolidar_csv(os.path.join(DATA_FOLDER, 'estaciones'), os.path.join(DATA_FOLDER, 'data.csv'))

In [ ]:
# Función para calcular 
def calcular_acumulados(df):
    # Asegurar que la columna Fecha está en formato de fecha
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    
    # Ordenar por CodigoEstacion y Fecha para asegurar el cálculo adecuado
    df = df.sort_values(by=['CodigoEstacion', 'Fecha'])
    
    # Definir los días para los que se calcularán promedios y sumatorias
    dias_promedios = [1, 3, 7, 15, 30]
    
    # Listas de columnas
    columnas_promedio = ['PTPM_CON','HR_CAL_MN_D', 'HR_CAL_MX_D', 'NV_MEDIA_D', 'NV_MN_D', 'NV_MX_D', 'TMN_CON', 'TMX_CON']
    
    # Crear un nuevo DataFrame para almacenar los resultados
    df_resultado = df.copy()
    
    # Calcular promedios móviles para cada estacion
    for dias in dias_promedios:
        for col in columnas_promedio:
            # Crear la columna nueva para el promedio
            col_nueva = f'{col}_{dias}D'
            # Calcular el promedio
            df_resultado[col_nueva] = df.groupby('CodigoEstacion')[col].transform(lambda x: x.shift(1).rolling(window=dias, min_periods=dias).mean())

    # Redondear a un decimal
    df_resultado = df_resultado.round(2)

    # Eliminar filas donde todas las columnas son NaN
    df_resultado = df_resultado.dropna(how='all', subset=[f'{col}_{dias}D' for dias in dias_promedios for col in columnas_promedio])

    return df_resultado

In [8]:
# Cargar el dataset
df = pd.read_csv(os.path.join(DATA_FOLDER, 'data.csv'))

# Calcular los acumulados
df_acumulados = calcular_acumulados(df)
df_acumulados = df_acumulados.drop(columns=['HR_CAL_MN_D', 'HR_CAL_MX_D', 'NV_MEDIA_D', 'NV_MN_D', 'NV_MX_D', 'TMN_CON', 'TMX_CON'])

# Guardar el resultado en un archivo CSV
output_path = os.path.join(DATA_FOLDER, 'data_acumulados.csv')
df_acumulados.to_csv(output_path, index=False)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17990978 entries, 0 to 17990977
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   CodigoEstacion  int64         
 1   Fecha           datetime64[ns]
 2   HR_CAL_MN_D     float64       
 3   HR_CAL_MX_D     float64       
 4   NV_MEDIA_D      float64       
 5   NV_MN_D         float64       
 6   NV_MX_D         float64       
 7   PTPM_CON        float64       
 8   TMN_CON         float64       
 9   TMX_CON         float64       
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 1.3 GB


In [13]:
df_acumulados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17988609 entries, 1 to 17743192
Data columns (total 43 columns):
 #   Column           Dtype         
---  ------           -----         
 0   CodigoEstacion   int64         
 1   Fecha            datetime64[ns]
 2   PTPM_CON         float64       
 3   PTPM_CON_1D      float64       
 4   HR_CAL_MN_D_1D   float64       
 5   HR_CAL_MX_D_1D   float64       
 6   NV_MEDIA_D_1D    float64       
 7   NV_MN_D_1D       float64       
 8   NV_MX_D_1D       float64       
 9   TMN_CON_1D       float64       
 10  TMX_CON_1D       float64       
 11  PTPM_CON_3D      float64       
 12  HR_CAL_MN_D_3D   float64       
 13  HR_CAL_MX_D_3D   float64       
 14  NV_MEDIA_D_3D    float64       
 15  NV_MN_D_3D       float64       
 16  NV_MX_D_3D       float64       
 17  TMN_CON_3D       float64       
 18  TMX_CON_3D       float64       
 19  PTPM_CON_7D      float64       
 20  HR_CAL_MN_D_7D   float64       
 21  HR_CAL_MX_D_7D   float64       
 2

## **Créditos**

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](https://www.linkedin.com/in/mario-andres-rodriguez-triana-394806145/).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*